In [80]:
import torch
import numpy as np
import pandas as pd
import transformers
from sklearn.model_selection import train_test_split
from transformers import AutoModel, BertTokenizerFast
from tqdm import tqdm
torch.cuda.is_available()

True

In [81]:
CVAP_all_SD_df = pd.read_csv('./ChineseEmoBank/CVAP_SD/CVAP_all_SD.csv', encoding= 'utf-8',sep="\t")
df = CVAP_all_SD_df.drop(['No.','Valence_SD', 'Arousal_SD'], axis= 1)
#df = df.iloc[:10,:]
#df = df.reset_index(drop = True)

df.loc[1]

Phrase           更加小心
Valence_Mean    5.111
Arousal_Mean    7.188
Name: 1, dtype: object

In [82]:

# 提取特徵和標籤
#X = df[['Valence_Mean', 'Arousal_Mean']]
#y = df['Phrase'] # 如果您的數據集中有標籤列，請替換 'label_column_name' 為您的標籤列名稱
x = df['Phrase']
y = df[['Valence_Mean']] # 如果您的數據集中有標籤列，請替換 'label_column_name' 為您的標籤列名稱

# 將數據集分成訓練集和測試集，以 80:20 的比例分割
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
print(x_train)


2003     沒比較好
1173    極度不尊重
1755     可能不怕
1017     還沒放棄
1836     應該不願
        ...  
1638    本來超開心
1095     最沒效率
1130     非常不爽
1294    極度不樂觀
860      異常嚴格
Name: Phrase, Length: 1687, dtype: object


In [83]:
bert = AutoModel.from_pretrained('bert-base-chinese', return_dict=False)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

# from transformers import (
#   BertTokenizerFast,
#   AutoModel,
# )

# tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
# bert = AutoModel.from_pretrained('ckiplab/bert-base-chinese')

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [84]:
train_idx = x_train.dropna().index
test_idx = x_test.dropna().index

train_tokens = tokenizer.batch_encode_plus(x_train[train_idx].to_list(),
                                           max_length = 50,
                                           pad_to_max_length = True,
                                           truncation = True)
test_tokens = tokenizer.batch_encode_plus(x_test[test_idx].to_list(),
                                           max_length = 50,
                                           pad_to_max_length = True,
                                           truncation = True)
print(y_train['Valence_Mean'])
# y_train = y_train.reset_index(drop = True)
y_train.loc[0]

2003    4.063
1173    2.000
1755    5.643
1017    6.333
1836    3.938
        ...  
1638    5.340
1095    2.688
1130    2.333
1294    1.500
860     2.222
Name: Valence_Mean, Length: 1687, dtype: float64


c:\Program Files\Python\38\lib\site-packages\transformers\tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Valence_Mean    3.45
Name: 0, dtype: float64

In [85]:
train_seq = torch.tensor(train_tokens['input_ids'])
train_mask = torch.tensor(train_tokens['attention_mask'])
# print([i for i in y_train['Valence_Mean']])
train_y = torch.tensor([i for i in y_train['Valence_Mean']])

test_seq = torch.tensor(test_tokens['input_ids'])
test_mask = torch.tensor(test_tokens['attention_mask'])
test_y = torch.tensor([i for i in y_test['Valence_Mean']])

In [86]:
from torch.utils.data import TensorDataset, RandomSampler, DataLoader

In [87]:
train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
trainloader = DataLoader(train_data, 
                         sampler = train_sampler,
                         batch_size = 32)

test_data = TensorDataset(test_seq, test_mask, test_y)
test_sampler = RandomSampler(test_data)
testloader = DataLoader(test_data, 
                         sampler = test_sampler,
                         batch_size = 32)

In [88]:
for param in bert.parameters():
    param.requires_grad = False

In [89]:
from torch import nn
from transformers import AdamW
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight

In [90]:
class BertRegressor(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.fc1 = nn.Linear(768, 1)  # output one continuous value
    
    def forward(self, sent_id, mask):
        _, cls_hs = self.bert(sent_id, attention_mask=mask)
        return self.fc1(cls_hs).squeeze()  # remove the last dimension of size 1

In [91]:
model = BertRegressor(bert)
model = model.cuda()

In [92]:
optimizer = AdamW(model.parameters(), lr=1e-5)

c:\Program Files\Python\38\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [93]:
# def loss_fn(outputs, targets):
#     return F.mse_loss(outputs, targets)

In [94]:
criterion = nn.MSELoss()

In [95]:
from tqdm import tqdm

In [96]:
epochs = 10

for e in range(epochs):   
    train_loss = 0.0
    for batch in tqdm(trainloader):
        batch = [i.cuda() for i in batch]
        sent_id, masks, labels = batch

        optimizer.zero_grad()
        preds = model(sent_id, masks)
        loss = criterion(preds, labels)
        train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
    print(f'Epoch:{e+1}\t\tTraining Loss: {train_loss / len(trainloader)}')

100%|██████████| 53/53 [00:04<00:00, 12.96it/s]


Epoch:1		Training Loss: 25.638851381697744


100%|██████████| 53/53 [00:03<00:00, 13.27it/s]


Epoch:2		Training Loss: 22.605714438096534


100%|██████████| 53/53 [00:04<00:00, 13.13it/s]


...

Epoch:8		Training Loss: 9.016397332245448


100%|██████████| 53/53 [00:04<00:00, 13.05it/s]


Epoch:9		Training Loss: 7.496373689399575


100%|██████████| 53/53 [00:04<00:00, 13.02it/s]

Epoch:10		Training Loss: 6.243675663786115


In [97]:
pred_label = []
true_label = []
for batch in tqdm(testloader):
    batch = [i.cuda() for i in batch]
    sent_id, masks, labels = batch

    preds = model(sent_id, masks)
    #pred_label.extend(torch.argmax(preds, axis = 1).cpu())
    pred_label.extend(preds.cpu())
    true_label.extend(labels.cpu())

100%|██████████| 18/18 [00:01<00:00, 13.34it/s]


In [98]:
#在上面的代码中，我们首先定义了两个空列表pred_label和true_label来存储
#模型的预测标签和真实标签。然后，我们遍历测试集并对每个批次进行预测
#。将预测值和真实值添加到相应的列表中后，我们可以使用
#sklearn库中的mean_absolute_error函数来计算MAE。
#最后，我们将MAE打印出来。
from sklearn.metrics import mean_absolute_error

pred_label = []
true_label = []
for batch in tqdm(testloader):
    batch = [i.cuda() for i in batch]
    sent_id, masks, labels = batch

    preds = model(sent_id, masks)
    pred_label.extend(preds.detach().cpu().numpy())
    true_label.extend(labels.detach().cpu().numpy())

mae = mean_absolute_error(true_label, pred_label)
print(f'MAE: {mae}')

100%|██████████| 18/18 [00:01<00:00, 13.29it/s]

MAE: 1.8106414079666138


In [105]:
def predict_sentiment(sentence, model, tokenizer):
    encoded_sent = tokenizer.encode_plus(
        sentence,
        truncation=True,
        max_length=50,
        add_special_tokens=True,
        # pad_to_max_length=True,
        padding='longest',
        return_attention_mask=True,
        return_tensors='pt'
    )
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_ids = encoded_sent['input_ids'].to(device)
    attention_mask = encoded_sent['attention_mask'].to(device)

    with torch.no_grad():
        output = model(input_ids, attention_mask)

    return output.item()

In [106]:
sentence = "这个电影太棒了，我非常喜欢！"
score = predict_sentiment(sentence, model, tokenizer)
print(score)

2.644368886947632
